<h2>Initializing</h2>
<p>Import Libraries, add variables for attribute strings (to save '') and load data</p>

In [1]:
import pandas as pd
import numpy as np
from scipy.stats.stats import pearsonr
from sklearn.cluster import KMeans
from sklearn import preprocessing
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix 
from sklearn import tree
from sklearn.metrics import classification_report
from scipy import interp
from sklearn.metrics import accuracy_score
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import cross_validate
import math

fixed_acidity = 'fixed acidity'
volatile_acidity = 'volatile acidity'
citric_acid = 'citric acid'
residual_sugar = 'residual sugar'
chlorides = 'chlorides'
free_sulfur_dioxide = 'free sulfur dioxide'
total_sulfur_dioxide = 'total sulfur dioxide'
density = 'density'
ph = 'pH'
sulphates = 'sulphates'
alcohol = 'alcohol'
quality = 'quality'
qclass = 'qclass'
ratio_to_fixed = 'ratio_to_fixed'
ratio_to_volatile = 'ratio_to_volatile'
ph_acidity = 'ph_acidity'
ratio_to_ph = 'ratio_to_ph'

In [2]:
def confusion_matrix_report(y_true, y_pred):
    cm, labels = confusion_matrix(y_true, y_pred), unique_labels(y_true, y_pred)
    column_width = max([len(str(x)) for x in labels] + [5])  # 5 is value length
    report = " " * column_width + " " + "{:_^{}}".format("Prediction", column_width * len(labels))+ "\n"
    report += " " * column_width + " ".join(["{:>{}}".format(label, column_width) for label in labels]) + "\n"
    for i, label1 in enumerate(labels):
        report += "{:>{}}".format(label1, column_width) + " ".join(["{:{}d}".format(cm[i, j], column_width) for j in range(len(labels))]) + "\n"
    return report
def root_mean_squared_error(y_true, y_pred):
        return math.sqrt(np.mean((y_pred - y_true)**2))

<p>Import red and white dataset.<br/>
Split into input matrix (independent vars) and output vector (target vars).<br/>
</p>

<p>sort for kNN (https://scikit-learn.org/stable/modules/neighbors.html#unsupervised-nearest-neighbors) </p>

In [3]:
white = pd.read_csv("data/winequality-white1.csv")
red = pd.read_csv("data/winequality-red1.csv")
white.sort_values(by='quality',inplace=True)
red.sort_values(by='quality',inplace=True)
white_target = white[quality]
red_target = red[quality]
white_input = white.drop(quality,axis=1)
red_input = red.drop(quality,axis=1)

Add ratio of acidity to sugar as attribute.

In [4]:
#white[ph_acidity] = 7 - white[ph]
#red[ph_acidity] = 7 - red[ph]
#white_input[ratio_to_fixed]=white[residual_sugar]/white[fixed_acidity]
#white_input[ratio_to_volatile]=white[residual_sugar]/white[volatile_acidity]
#white_input[ratio_to_ph]=white[residual_sugar]/white[ph_acidity]
#red_input[ratio_to_fixed]=red[residual_sugar]/red[fixed_acidity]
#red_input[ratio_to_volatile]=red[residual_sugar]/red[volatile_acidity]
#red_input[ratio_to_ph]=red[residual_sugar]/red[ph_acidity]

<p>using log. (and filtering based on results of naive bayes)</p>

In [5]:
log_white_norm=np.log(white_input+1)
log_red_norm=np.log(red_input+1)
f_log_white=log_white_norm.drop([fixed_acidity,sulphates,chlorides,citric_acid,ph,density,total_sulfur_dioxide,residual_sugar],axis=1)
f_log_red=log_red_norm.drop([free_sulfur_dioxide,ph,residual_sugar,chlorides,citric_acid,fixed_acidity,density],axis=1)

<p>Normalize using MinMax Normalizer. (and filtering based on results of naive bayes)</p>

In [6]:
mm_white_norm=(white_input-white_input.min())/(white_input.max()-white_input.min())
mm_red_norm=(red_input-red_input.min())/(red_input.max()-red_input.min())
f_mm_white = mm_white_norm.drop([fixed_acidity,sulphates,chlorides,citric_acid,ph,density,total_sulfur_dioxide,residual_sugar],axis=1)
f_mm_red = mm_red_norm.drop([free_sulfur_dioxide,ph,residual_sugar,chlorides,citric_acid,fixed_acidity,density],axis=1)

<p>Normalize using Std. Normalizer. (and filtering based on results of naive bayes)</p>

In [7]:
white_norm=(white_input-white_input.mean())/(white_input.std())
red_norm=(red_input-red_input.mean())/(red_input.std())
f_white_norm=white_norm.drop([fixed_acidity,sulphates,chlorides,citric_acid,ph,density,total_sulfur_dioxide,residual_sugar],axis=1)
f_red_norm=red_norm.drop([free_sulfur_dioxide,ph,residual_sugar,chlorides,citric_acid,fixed_acidity,density],axis=1)

<p>Correlation Matrices</p>

In [8]:
#wcorr = white.corr()
#wcorr.style.background_gradient().set_precision(3)

In [9]:
#rcorr = red.corr()
#rcorr.style.background_gradient().set_precision(3)

<h2>Assigning classes</h2>
<p>
    Assign classes based on quality. Less than 6; 6; better than 6. Because it makes sense considering the distribution of ratings.
</p>

In [10]:
#Assign classes for white
conditions = [(white['quality'] < 6),
             (white['quality'] > 6)]
choices = [1,3]
white['classnum'] = np.select(conditions, choices, default = 2)
white_classnum = white['classnum']
white.drop('classnum',axis=1,inplace=True)
#choices = ['2_low', '1_high']
#white['class'] = np.select(conditions, choices, default = '3_medium')
#choices = [1,3]
#white['classnum'] = np.select(conditions, choices, default = 2)
#white_classnum = white['classnum']
#white.drop('classnum',axis=1,inplace=True)
#Assign classes for red
conditions = [(red['quality'] < 6),
              (red['quality'] > 6)]
#choices = ['3_low', '1_high']
#red['class'] = np.select(conditions, choices, default = '2_medium')
#choices = [1,3]
red['classnum'] = np.select(conditions, choices, default = 2)
red_classnum = red['classnum']
red.drop('classnum',axis=1,inplace=True)

#print('class distribution fot white:')
#print(white['class'].value_counts())
#print('\n class distribution for red:')
#print(red['class'].value_counts())

#white_norm['class']=white['class']
#red_norm['class']=red['class']

white_targetclass = white_classnum
red_targetclass = red_classnum
print(white_targetclass.value_counts())
print(red_targetclass.value_counts())
#white_targetclass = white['class']
#red_targetclass = red['class']

2    2198
1    1640
3    1060
Name: classnum, dtype: int64
1    744
2    638
3    217
Name: classnum, dtype: int64


In [11]:
white_norm_input = white_norm
red_norm_input = red_norm
print(white_norm_input.head())
print(red_norm_input.head())

      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
3307       3.016125         -0.379397    -0.365159        0.415726  -0.401518   
445        0.290581          0.414297    -0.117266        0.908622  -0.355747   
2050       5.860171         -0.478608     0.378521        0.928338  -0.538831   
3810      -0.064924         -0.180973     0.047996        1.716973   0.651211   
3409      -0.775936         -0.478608     0.130627       -1.122112   0.239274   

      free sulfur dioxide  total sulfur dioxide   density        pH  \
3307             5.214982              1.638647 -0.025871 -1.909043   
445             -1.135293             -1.702681 -0.109457  0.342604   
2050            -1.194092             -0.361444  1.896623 -1.710368   
3810             0.393477              0.556245  1.010604  0.342604   
3409            -0.664902             -0.643809 -0.811585  1.203527   

      sulphates   alcohol  
3307  -0.962507  0.394706  
445   -0.787262  0.801005  
20

In [12]:
#white_norm.loc[white_norm['class']=='3_high'].describe()
#white_norm.loc[white_norm['class']=='2_medium'].describe()
#white_norm.loc[white_norm['class']=='1_low'].describe()
white_norm_filtered = white_norm_input.drop([fixed_acidity,sulphates,chlorides,citric_acid,ph,density,total_sulfur_dioxide,residual_sugar],axis=1)
white_norm_filtered.describe()

,volatile acidity,free sulfur dioxide,alcohol
count,4.898000e+03,4.898000e+03,4.898000e+03
mean,-1.998539e-14,-6.380722e-17,-4.325046e-14
std,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.966784e+00,-1.958477e+00,-2.043089e+00
25%,-6.770318e-01,-7.237012e-01,-8.241915e-01
50%,-1.809733e-01,-7.691388e-02,-9.285319e-02
75%,4.142970e-01,6.286722e-01,7.197450e-01
max,8.152811e+00,1.491679e+01,2.995020e+00


In [13]:
#red_norm.loc[red_norm['class']=='3_high'].describe()
#red_norm.loc[red_norm['class']=='2_medium'].describe()
#red_norm.loc[red_norm['class']=='1_low'].describe()
red_norm_filtered = red_norm_input.drop([free_sulfur_dioxide,ph,residual_sugar,chlorides,citric_acid,fixed_acidity,density],axis=1)
red_norm_filtered.describe()

,volatile acidity,total sulfur dioxide,sulphates,alcohol
count,1.599000e+03,1.599000e+03,1.599000e+03,1.599000e+03
mean,8.477896e-15,2.365560e-16,6.625737e-15,2.204136e-14
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.277567e+00,-1.230199e+00,-1.935902e+00,-1.898325e+00
25%,-7.696903e-01,-7.438076e-01,-6.380200e-01,-8.661079e-01
50%,-4.367545e-02,-2.574163e-01,-2.250577e-01,-2.092427e-01
75%,6.264921e-01,4.721707e-01,4.238832e-01,6.352984e-01
max,5.876138e+00,7.372847e+00,7.916200e+00,4.201138e+00


<h2>k-NN classification<h2>

In [14]:
white_norm_filtered.describe()

,volatile acidity,free sulfur dioxide,alcohol
count,4.898000e+03,4.898000e+03,4.898000e+03
mean,-1.998539e-14,-6.380722e-17,-4.325046e-14
std,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.966784e+00,-1.958477e+00,-2.043089e+00
25%,-6.770318e-01,-7.237012e-01,-8.241915e-01
50%,-1.809733e-01,-7.691388e-02,-9.285319e-02
75%,4.142970e-01,6.286722e-01,7.197450e-01
max,8.152811e+00,1.491679e+01,2.995020e+00


<h3>white: knn normalized and filtered based on Naive Bayes results</h3>
RMSE/P/R/F1
<p>White,STD Normed, filteredNB:0.68/0.66/0.66/0.66 at k=1, decreasing after<br>
White log normed, filteredNB: 0.69/0.64/0.64/0.64 at k=1, decreasing after<br/>
White MM normed, filteredNB: 0.68/0.65/0.65/0.65 at k=1, decreasing after</p>

In [15]:
def run_knn_report(inputs, targets):
    print("_______________________________________________________________________________________________")
    print("KNN, no weights")
    for n_neighbour in range(1,21):
        print(str(n_neighbour) + " neighbours:")
        knn_estimator = KNeighborsClassifier(n_neighbour)
        predicted = cross_val_predict(knn_estimator,inputs,targets,cv=cv)
        print(confusion_matrix_report(targets,predicted))
        print(classification_report(targets,predicted))
        try:
            print("RMSE:")
            print(root_mean_squared_error(targets,predicted))
        except(Error):
            pass
    
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1337)
#cv = KFold(n_splits=3)
print("#____________________white STD normed filtered____________________#")
run_knn_report(f_white_norm.values, white_targetclass)

#____________________white STD normed filtered____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1 1120   419   101
    2  420  1444   334
    3   84   326   650

             precision    recall  f1-score   support

          1       0.69      0.68      0.69      1640
          2       0.66      0.66      0.66      2198
          3       0.60      0.61      0.61      1060

avg / total       0.66      0.66      0.66      4898

RMSE:
0.6761104623422782
2 neighbours:
      __Prediction___
         1     2     3
    1 1310   286    44
    2  799  1243   156
    3  197   484   379

             precision    recall  f1-score   support

          1       0.57      0.80      0.66      1640
          2       0.62      0.57      0.59      2198
          3       0.65      0.36      0.46      1060

avg / total       0.61      0.60      0.59      4898

R

             precision    recall  f1-score   support

          1       0.63      0.63      0.63      1640
          2       0.55      0.64      0.59      2198
          3       0.58      0.37      0.45      1060

avg / total       0.58      0.58      0.58      4898

RMSE:
0.6938450750408423


In [16]:
print("#____________________white log normed filtered____________________#")
run_knn_report(f_log_white.values, white_targetclass)

#____________________white log normed filtered____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1 1081   461    98
    2  439  1413   346
    3   98   327   635

             precision    recall  f1-score   support

          1       0.67      0.66      0.66      1640
          2       0.64      0.64      0.64      2198
          3       0.59      0.60      0.59      1060

avg / total       0.64      0.64      0.64      4898

RMSE:
0.6936979336808926
2 neighbours:
      __Prediction___
         1     2     3
    1 1278   319    43
    2  810  1228   160
    3  199   520   341

             precision    recall  f1-score   support

          1       0.56      0.78      0.65      1640
          2       0.59      0.56      0.58      2198
          3       0.63      0.32      0.43      1060

avg / total       0.59      0.58      0.57      4898

R

      __Prediction___
         1     2     3
    1 1010   579    51
    2  555  1382   261
    3   65   612   383

             precision    recall  f1-score   support

          1       0.62      0.62      0.62      1640
          2       0.54      0.63      0.58      2198
          3       0.55      0.36      0.44      1060

avg / total       0.57      0.57      0.56      4898

RMSE:
0.7102757416922127


In [17]:
print("#____________________white MinMax normed filtered____________________#")
run_knn_report(f_mm_white.values, white_targetclass)

#____________________white MinMax normed filtered____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1 1118   418   104
    2  436  1423   339
    3   79   336   645

             precision    recall  f1-score   support

          1       0.68      0.68      0.68      1640
          2       0.65      0.65      0.65      2198
          3       0.59      0.61      0.60      1060

avg / total       0.65      0.65      0.65      4898

RMSE:
0.6794240108541295
2 neighbours:
      __Prediction___
         1     2     3
    1 1281   321    38
    2  795  1243   160
    3  183   490   387

             precision    recall  f1-score   support

          1       0.57      0.78      0.66      1640
          2       0.61      0.57      0.58      2198
          3       0.66      0.37      0.47      1060

avg / total       0.60      0.59      0.58      4898

<h3><i>red: </i>knn normalized and filtered based on Naive Bayes results</h3>
<p>
    STD,NBF: 0.6/0.7/0.7/0.7 at k=1, decreasing after<br>
    MM,NBF: 0.6/0.7/0.7/0.7 at k=1, decreasing after<br>
    log NBF: 0.59/0.69/0.69/0.69 at k=1, decreasing after<br>
</p>

In [18]:
print("#____________________red STD normed____________________#")
run_knn_report(f_red_norm.values, red_targetclass)

#____________________red STD normed____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1  567   165    12
    2  147   417    74
    3   18    69   130

             precision    recall  f1-score   support

          1       0.77      0.76      0.77       744
          2       0.64      0.65      0.65       638
          3       0.60      0.60      0.60       217

avg / total       0.70      0.70      0.70      1599

RMSE:
0.5996663654430289
2 neighbours:
      __Prediction___
         1     2     3
    1  641   100     3
    2  288   322    28
    3   36   121    60

             precision    recall  f1-score   support

          1       0.66      0.86      0.75       744
          2       0.59      0.50      0.55       638
          3       0.66      0.28      0.39       217

avg / total       0.64      0.64      0.62      1599

RMSE:
0.6583

In [19]:
print("#____________________red MinMax normed____________________#")
run_knn_report(f_mm_red.values, red_targetclass)

#____________________red MinMax normed____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1  567   163    14
    2  142   420    76
    3   15    66   136

             precision    recall  f1-score   support

          1       0.78      0.76      0.77       744
          2       0.65      0.66      0.65       638
          3       0.60      0.63      0.61       217

avg / total       0.70      0.70      0.70      1599

RMSE:
0.5933759848629978
2 neighbours:
      __Prediction___
         1     2     3
    1  639   103     2
    2  281   325    32
    3   34   125    58

             precision    recall  f1-score   support

          1       0.67      0.86      0.75       744
          2       0.59      0.51      0.55       638
          3       0.63      0.27      0.38       217

avg / total       0.63      0.64      0.62      1599

RMSE:
0.6

In [20]:
print("#____________________red log normed____________________#")
run_knn_report(f_log_red.values, red_targetclass)

#____________________red log normed____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1  568   160    16
    2  154   402    82
    3   16    66   135

             precision    recall  f1-score   support

          1       0.77      0.76      0.77       744
          2       0.64      0.63      0.64       638
          3       0.58      0.62      0.60       217

avg / total       0.69      0.69      0.69      1599

RMSE:
0.6074377440388866
2 neighbours:
      __Prediction___
         1     2     3
    1  646    92     6
    2  292   314    32
    3   37   119    61

             precision    recall  f1-score   support

          1       0.66      0.87      0.75       744
          2       0.60      0.49      0.54       638
          3       0.62      0.28      0.39       217

avg / total       0.63      0.64      0.62      1599

RMSE:
0.6649

<h3>*white: normalized unfiltered</h3>
<p>
    STD: 0.62/0.7/0.7/0.7 at k=1, decreasing after<br>
    MM: 0.62/0.7/0.7/0.7 at k=1, decreasing after<br>
    log: 0.65/0.67/0.67/0.67 at k=1, decreasing after<br>
</p>

In [21]:
print("#____________________white STD normed____________________#")
run_knn_report(white_norm.values, white_targetclass)

#____________________white STD normed____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1 1156   416    68
    2  381  1538   279
    3   59   276   725

             precision    recall  f1-score   support

          1       0.72      0.70      0.71      1640
          2       0.69      0.70      0.69      2198
          3       0.68      0.68      0.68      1060

avg / total       0.70      0.70      0.70      4898

RMSE:
0.6162360225133208
2 neighbours:
      __Prediction___
         1     2     3
    1 1308   303    29
    2  735  1331   132
    3  132   445   483

             precision    recall  f1-score   support

          1       0.60      0.80      0.69      1640
          2       0.64      0.61      0.62      2198
          3       0.75      0.46      0.57      1060

avg / total       0.65      0.64      0.63      4898

RMSE:
0.67

      __Prediction___
         1     2     3
    1 1039   548    53
    2  514  1406   278
    3   60   506   494

             precision    recall  f1-score   support

          1       0.64      0.63      0.64      1640
          2       0.57      0.64      0.60      2198
          3       0.60      0.47      0.52      1060

avg / total       0.60      0.60      0.60      4898

RMSE:
0.6849606486805754


In [22]:
print("#____________________white MinMax normed____________________#")
run_knn_report(mm_white_norm.values, white_targetclass)

#____________________white MinMax normed____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1 1163   408    69
    2  375  1528   295
    3   60   277   723

             precision    recall  f1-score   support

          1       0.73      0.71      0.72      1640
          2       0.69      0.70      0.69      2198
          3       0.67      0.68      0.67      1060

avg / total       0.70      0.70      0.70      4898

RMSE:
0.6180555396250805
2 neighbours:
      __Prediction___
         1     2     3
    1 1324   282    34
    2  740  1317   141
    3  133   461   466

             precision    recall  f1-score   support

          1       0.60      0.81      0.69      1640
          2       0.64      0.60      0.62      2198
          3       0.73      0.44      0.55      1060

avg / total       0.65      0.63      0.63      4898

RMSE:
0

      __Prediction___
         1     2     3
    1 1022   552    66
    2  531  1332   335
    3   49   500   511

             precision    recall  f1-score   support

          1       0.64      0.62      0.63      1640
          2       0.56      0.61      0.58      2198
          3       0.56      0.48      0.52      1060

avg / total       0.59      0.58      0.58      4898

RMSE:
0.6967813770934691


In [23]:
print("#____________________white log normed____________________#")
run_knn_report(log_white_norm.values, white_targetclass)

#____________________white log normed____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1 1139   412    89
    2  416  1464   318
    3   67   304   689

             precision    recall  f1-score   support

          1       0.70      0.69      0.70      1640
          2       0.67      0.67      0.67      2198
          3       0.63      0.65      0.64      1060

avg / total       0.67      0.67      0.67      4898

RMSE:
0.6507212444783992
2 neighbours:
      __Prediction___
         1     2     3
    1 1280   322    38
    2  771  1281   146
    3  160   480   420

             precision    recall  f1-score   support

          1       0.58      0.78      0.66      1640
          2       0.61      0.58      0.60      2198
          3       0.70      0.40      0.50      1060

avg / total       0.62      0.61      0.60      4898

RMSE:
0.71

             precision    recall  f1-score   support

          1       0.61      0.59      0.60      1640
          2       0.55      0.64      0.59      2198
          3       0.56      0.39      0.46      1060

avg / total       0.57      0.57      0.57      4898

RMSE:
0.7205494478087149


<h3><i>*red: </i> normalized unfiltered</h3>
<p>
    STD: 0.63/0.69/0.69/0.69. k=1 more=bad<br>
    MM: 0.63/0.69/0.69/0.69. k=1 more=bad<br>
    log: 0.63/0.68/0.68/0.68. k=1 more=bad<br>
</p>

In [24]:
print("#____________________red STD normed____________________#")
run_knn_report(red_norm.values, red_targetclass)

#____________________red STD normed____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1  554   166    24
    2  158   412    68
    3   21    61   135

             precision    recall  f1-score   support

          1       0.76      0.74      0.75       744
          2       0.64      0.65      0.65       638
          3       0.59      0.62      0.61       217

avg / total       0.69      0.69      0.69      1599

RMSE:
0.6291839319806762
2 neighbours:
      __Prediction___
         1     2     3
    1  627   109     8
    2  286   316    36
    3   39   105    73

             precision    recall  f1-score   support

          1       0.66      0.84      0.74       744
          2       0.60      0.50      0.54       638
          3       0.62      0.34      0.44       217

avg / total       0.63      0.64      0.62      1599

RMSE:
0.6728

In [25]:
print("#____________________red MinMax normed____________________#")
run_knn_report(mm_red_norm.values, red_targetclass)

#____________________red MinMax normed____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1  553   167    24
    2  155   416    67
    3   21    66   130

             precision    recall  f1-score   support

          1       0.76      0.74      0.75       744
          2       0.64      0.65      0.65       638
          3       0.59      0.60      0.59       217

avg / total       0.69      0.69      0.69      1599

RMSE:
0.630177119547553
2 neighbours:
      __Prediction___
         1     2     3
    1  625   116     3
    2  292   312    34
    3   44    97    76

             precision    recall  f1-score   support

          1       0.65      0.84      0.73       744
          2       0.59      0.49      0.54       638
          3       0.67      0.35      0.46       217

avg / total       0.63      0.63      0.62      1599

RMSE:
0.67

In [26]:
print("#____________________red log normed____________________#")
run_knn_report(log_red_norm.values, red_targetclass)

#____________________red log normed____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1  558   163    23
    2  168   395    75
    3   15    70   132

             precision    recall  f1-score   support

          1       0.75      0.75      0.75       744
          2       0.63      0.62      0.62       638
          3       0.57      0.61      0.59       217

avg / total       0.68      0.68      0.68      1599

RMSE:
0.6266940768159515
2 neighbours:
      __Prediction___
         1     2     3
    1  630   105     9
    2  310   293    35
    3   48   105    64

             precision    recall  f1-score   support

          1       0.64      0.85      0.73       744
          2       0.58      0.46      0.51       638
          3       0.59      0.29      0.39       217

avg / total       0.61      0.62      0.60      1599

RMSE:
0.6997

<h3> white: unfiltered, unnormalized <h3>

In [27]:
print("#____________________white_____________________#")
run_knn_report(white_input.values,white_targetclass)

#____________________white_____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1 1052   450   138
    2  437  1435   326
    3  107   325   628

             precision    recall  f1-score   support

          1       0.66      0.64      0.65      1640
          2       0.65      0.65      0.65      2198
          3       0.58      0.59      0.58      1060

avg / total       0.64      0.64      0.64      4898

RMSE:
0.716998872108698
2 neighbours:
      __Prediction___
         1     2     3
    1 1234   345    61
    2  856  1200   142
    3  265   467   328

             precision    recall  f1-score   support

          1       0.52      0.75      0.62      1640
          2       0.60      0.55      0.57      2198
          3       0.62      0.31      0.41      1060

avg / total       0.58      0.56      0.55      4898

RMSE:
0.7973516802008

      __Prediction___
         1     2     3
    1  852   701    87
    2  632  1339   227
    3  195   600   265

             precision    recall  f1-score   support

          1       0.51      0.52      0.51      1640
          2       0.51      0.61      0.55      2198
          3       0.46      0.25      0.32      1060

avg / total       0.50      0.50      0.49      4898

RMSE:
0.819325579900903


<p>
    unnormed white: 0.71/0.64/0.64/0.64, k=1 more bad
</p>

<h3>red, unnormalized, unselected</h3>

In [28]:
print("#____________________red_____________________#")
run_knn_report(white_input.values,white_targetclass)

#____________________red_____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1 1052   450   138
    2  437  1435   326
    3  107   325   628

             precision    recall  f1-score   support

          1       0.66      0.64      0.65      1640
          2       0.65      0.65      0.65      2198
          3       0.58      0.59      0.58      1060

avg / total       0.64      0.64      0.64      4898

RMSE:
0.716998872108698
2 neighbours:
      __Prediction___
         1     2     3
    1 1234   345    61
    2  856  1200   142
    3  265   467   328

             precision    recall  f1-score   support

          1       0.52      0.75      0.62      1640
          2       0.60      0.55      0.57      2198
          3       0.62      0.31      0.41      1060

avg / total       0.58      0.56      0.55      4898

RMSE:
0.797351680200870

      __Prediction___
         1     2     3
    1  852   701    87
    2  632  1339   227
    3  195   600   265

             precision    recall  f1-score   support

          1       0.51      0.52      0.51      1640
          2       0.51      0.61      0.55      2198
          3       0.46      0.25      0.32      1060

avg / total       0.50      0.50      0.49      4898

RMSE:
0.819325579900903


0.72 0.64      0.64      0.64


In [29]:
def run_knc_report(inputs, targets):
    knc_estimator = NearestCentroid()
    predicted = cross_val_predict(knc_estimator,inputs,targets,cv=cv)
    print(confusion_matrix_report(targets,predicted))
    print(classification_report(targets,predicted))
    try:
        print("RMSE:")
        print(root_mean_squared_error(targets,predicted))
    except(Error):
        pass

In [30]:
print("#____________________white STD normed____________________#")
run_knc_report(white_norm.values, white_targetclass)

#____________________white STD normed____________________#
      __Prediction___
         1     2     3
    1 1081   266   293
    2  785   537   876
    3  139   181   740

             precision    recall  f1-score   support

          1       0.54      0.66      0.59      1640
          2       0.55      0.24      0.34      2198
          3       0.39      0.70      0.50      1060

avg / total       0.51      0.48      0.46      4898

RMSE:
0.8849727718184005


In [31]:
print("#____________________white MinMax normed____________________#")
run_knc_report(mm_white_norm.values, white_targetclass)

#____________________white MinMax normed____________________#
      __Prediction___
         1     2     3
    1 1143   297   200
    2  849   552   797
    3  145   194   721

             precision    recall  f1-score   support

          1       0.53      0.70      0.61      1640
          2       0.53      0.25      0.34      2198
          3       0.42      0.68      0.52      1060

avg / total       0.51      0.49      0.47      4898

RMSE:
0.8473772376762365


In [32]:

print("#____________________white log normed____________________#")
run_knc_report(log_white_norm.values, white_targetclass)

#____________________white log normed____________________#
      __Prediction___
         1     2     3
    1  893   126   621
    2  949   274   975
    3  306   143   611

             precision    recall  f1-score   support

          1       0.42      0.54      0.47      1640
          2       0.50      0.12      0.20      2198
          3       0.28      0.58      0.37      1060

avg / total       0.43      0.36      0.33      4898

RMSE:
1.097623551217735


In [33]:
print("#____________________red STD normed____________________#")
run_knc_report(red_norm.values, red_targetclass)

#____________________red STD normed____________________#
      __Prediction___
         1     2     3
    1  541   145    58
    2  217   220   201
    3   11    43   163

             precision    recall  f1-score   support

          1       0.70      0.73      0.72       744
          2       0.54      0.34      0.42       638
          3       0.39      0.75      0.51       217

avg / total       0.59      0.58      0.57      1599

RMSE:
0.7426942484742277


In [34]:
print("#____________________red MinMax normed____________________#")
run_knc_report(mm_red_norm.values, red_targetclass)

#____________________red MinMax normed____________________#
      __Prediction___
         1     2     3
    1  559   113    72
    2  243   189   206
    3   12    44   161

             precision    recall  f1-score   support

          1       0.69      0.75      0.72       744
          2       0.55      0.30      0.38       638
          3       0.37      0.74      0.49       217

avg / total       0.59      0.57      0.55      1599

RMSE:
0.7675403565118231


In [35]:
print("#____________________red log normed____________________#")
run_knc_report(log_red_norm.values, red_targetclass)

#____________________red log normed____________________#
      __Prediction___
         1     2     3
    1  430    81   233
    2  289   100   249
    3   66    23   128

             precision    recall  f1-score   support

          1       0.55      0.58      0.56       744
          2       0.49      0.16      0.24       638
          3       0.21      0.59      0.31       217

avg / total       0.48      0.41      0.40      1599

RMSE:
1.0721326493308097


In [36]:
def run_kncs_report(inputs, targets,thresh=0):
    knc_estimator = NearestCentroid(shrink_threshold=thresh)
    predicted = cross_val_predict(knc_estimator,inputs,targets,cv=cv)
    print(confusion_matrix_report(targets,predicted))
    print(classification_report(targets,predicted))
    try:
        print("RMSE:")
        print(root_mean_squared_error(targets,predicted))
    except(Error):
        pass

In [37]:
for t in range (0,10000,100):
    t = t/1000
    print(t)
    print("#____________________white STD normed____________________#")
    run_kncs_report(white_norm.values, white_targetclass,t)
    print("#____________________white MinMax normed____________________#")
    run_kncs_report(mm_white_norm.values, white_targetclass,t)
    print("#____________________white log normed____________________#")
    run_kncs_report(log_white_norm.values, white_targetclass,t)

0.0
#____________________white STD normed____________________#
      __Prediction___
         1     2     3
    1 1081   266   293
    2  785   537   876
    3  139   181   740

             precision    recall  f1-score   support

          1       0.54      0.66      0.59      1640
          2       0.55      0.24      0.34      2198
          3       0.39      0.70      0.50      1060

avg / total       0.51      0.48      0.46      4898

RMSE:
0.8849727718184005
#____________________white MinMax normed____________________#
      __Prediction___
         1     2     3
    1 1143   297   200
    2  849   552   797
    3  145   194   721

             precision    recall  f1-score   support

          1       0.53      0.70      0.61      1640
          2       0.53      0.25      0.34      2198
          3       0.42      0.68      0.52      1060

avg / total       0.51      0.49      0.47      4898

RMSE:
0.8473772376762365
#____________________white log normed____________________#


             precision    recall  f1-score   support

          1       0.54      0.66      0.60      1640
          2       0.55      0.24      0.33      2198
          3       0.39      0.71      0.51      1060

avg / total       0.51      0.48      0.46      4898

RMSE:
0.8824314010865206
#____________________white MinMax normed____________________#
      __Prediction___
         1     2     3
    1 1149   286   205
    2  851   530   817
    3  151   184   725

             precision    recall  f1-score   support

          1       0.53      0.70      0.61      1640
          2       0.53      0.24      0.33      2198
          3       0.41      0.68      0.52      1060

avg / total       0.51      0.49      0.46      4898

RMSE:
0.852781101086291
#____________________white log normed____________________#
      __Prediction___
         1     2     3
    1  914   101   625
    2  981   240   977
    3  310   131   619

             precision    recall  f1-score   support

          

0.886240724339699
#____________________white MinMax normed____________________#
      __Prediction___
         1     2     3
    1 1152   276   212
    2  854   522   822
    3  150   183   727

             precision    recall  f1-score   support

          1       0.53      0.70      0.61      1640
          2       0.53      0.24      0.33      2198
          3       0.41      0.69      0.52      1060

avg / total       0.51      0.49      0.46      4898

RMSE:
0.8552912197848626
#____________________white log normed____________________#
      __Prediction___
         1     2     3
    1  929    74   637
    2 1001   209   988
    3  306   126   628

             precision    recall  f1-score   support

          1       0.42      0.57      0.48      1640
          2       0.51      0.10      0.16      2198
          3       0.28      0.59      0.38      1060

avg / total       0.43      0.36      0.31      4898

RMSE:
1.103189629259335
1.3
#____________________white STD normed_____

             precision    recall  f1-score   support

          1       0.54      0.71      0.61      1640
          2       0.53      0.23      0.32      2198
          3       0.41      0.68      0.51      1060

avg / total       0.51      0.49      0.46      4898

RMSE:
0.8588643778850282
#____________________white log normed____________________#
      __Prediction___
         1     2     3
    1  936    60   644
    2 1016   181  1001
    3  304   125   631

             precision    recall  f1-score   support

          1       0.41      0.57      0.48      1640
          2       0.49      0.08      0.14      2198
          3       0.28      0.60      0.38      1060

avg / total       0.42      0.36      0.31      4898

RMSE:
1.106239034729829
1.9
#____________________white STD normed____________________#
      __Prediction___
         1     2     3
    1 1098   244   298
    2  823   462   913
    3  142   158   760

             precision    recall  f1-score   support

         

             precision    recall  f1-score   support

          1       0.53      0.71      0.61      1640
          2       0.53      0.22      0.31      2198
          3       0.41      0.69      0.51      1060

avg / total       0.50      0.49      0.46      4898

RMSE:
0.8610011550370255
#____________________white log normed____________________#
      __Prediction___
         1     2     3
    1  946    47   647
    2 1016   169  1013
    3  301   112   647

             precision    recall  f1-score   support

          1       0.42      0.58      0.48      1640
          2       0.52      0.08      0.13      2198
          3       0.28      0.61      0.38      1060

avg / total       0.43      0.36      0.31      4898

RMSE:
1.1049463753711743
2.5
#____________________white STD normed____________________#
      __Prediction___
         1     2     3
    1 1111   229   300
    2  842   439   917
    3  147   157   756

             precision    recall  f1-score   support

        

...

             precision    recall  f1-score   support

          1       0.51      0.73      0.60      1640
          2       0.52      0.15      0.24      2198
          3       0.40      0.72      0.52      1060

avg / total       0.49      0.47      0.42      4898

RMSE:
0.8912944967579461
#____________________white log normed____________________#
      __Prediction___
         1     2     3
    1 1168   183   289
    2  972   272   954
    3  184    77   799

             precision    recall  f1-score   support

          1       0.50      0.71      0.59      1640
          2       0.51      0.12      0.20      2198
          3       0.39      0.75      0.52      1060

avg / total       0.48      0.46      0.40      4898

RMSE:
0.912460809273799
7.3
#____________________white STD normed____________________#
      __Prediction___
         1     2     3
    1 1224   121   295
    2 1024   213   961
    3  192    78   790

             precision    recall  f1-score   support

         

             precision    recall  f1-score   support

          1       0.51      0.74      0.60      1640
          2       0.52      0.13      0.21      2198
          3       0.40      0.73      0.51      1060

avg / total       0.49      0.46      0.41      4898

RMSE:
0.9001837297150915
#____________________white log normed____________________#
      __Prediction___
         1     2     3
    1 1185   165   290
    2  980   263   955
    3  187    73   800

             precision    recall  f1-score   support

          1       0.50      0.72      0.59      1640
          2       0.52      0.12      0.19      2198
          3       0.39      0.75      0.52      1060

avg / total       0.49      0.46      0.40      4898

RMSE:
0.9127963756262796
7.9
#____________________white STD normed____________________#
      __Prediction___
         1     2     3
    1 1228   118   294
    2 1050   183   965
    3  201    69   790

             precision    recall  f1-score   support

        

             precision    recall  f1-score   support

          1       0.50      0.74      0.60      1640
          2       0.51      0.12      0.20      2198
          3       0.40      0.73      0.51      1060

avg / total       0.48      0.46      0.40      4898

RMSE:
0.9050468721294644
#____________________white log normed____________________#
      __Prediction___
         1     2     3
    1 1213   137   290
    2 1005   238   955
    3  197    63   800

             precision    recall  f1-score   support

          1       0.50      0.74      0.60      1640
          2       0.54      0.11      0.18      2198
          3       0.39      0.75      0.52      1060

avg / total       0.50      0.46      0.39      4898

RMSE:
0.915810939779251
8.5
#____________________white STD normed____________________#
      __Prediction___
         1     2     3
    1 1237   111   292
    2 1064   168   966
    3  205    60   795

             precision    recall  f1-score   support

         

      __Prediction___
         1     2     3
    1 1214   136   290
    2 1009   234   955
    3  197    63   800

             precision    recall  f1-score   support

          1       0.50      0.74      0.60      1640
          2       0.54      0.11      0.18      2198
          3       0.39      0.75      0.52      1060

avg / total       0.50      0.46      0.39      4898

RMSE:
0.9161452790443395
9.1
#____________________white STD normed____________________#
      __Prediction___
         1     2     3
    1 1243   107   290
    2 1079   151   968
    3  209    56   795

             precision    recall  f1-score   support

          1       0.49      0.76      0.60      1640
          2       0.48      0.07      0.12      2198
          3       0.39      0.75      0.51      1060

avg / total       0.46      0.45      0.36      4898

RMSE:
0.9266702995229568
#____________________white MinMax normed____________________#
      __Prediction___
         1     2     3
    1 1214   1

             precision    recall  f1-score   support

          1       0.50      0.74      0.60      1640
          2       0.53      0.09      0.15      2198
          3       0.38      0.76      0.51      1060

avg / total       0.49      0.45      0.38      4898

RMSE:
0.925788595898454
9.7
#____________________white STD normed____________________#
      __Prediction___
         1     2     3
    1 1259    88   293
    2 1092   154   952
    3  222    38   800

             precision    recall  f1-score   support

          1       0.49      0.77      0.60      1640
          2       0.55      0.07      0.12      2198
          3       0.39      0.75      0.52      1060

avg / total       0.50      0.45      0.37      4898

RMSE:
0.9293103911960597
#____________________white MinMax normed____________________#
      __Prediction___
         1     2     3
    1 1214   166   260
    2 1009   272   917
    3  197    92   771

             precision    recall  f1-score   support

      

In [38]:
for t in range (0,10000,100):
    t = t/1000
    print(t)
    run_kncs_report(white_norm_input.values,white_targetclass,t)
    run_kncs_report(red_norm_input.values,red_targetclass,t)    

0.0
      __Prediction___
         1     2     3
    1 1081   266   293
    2  785   537   876
    3  139   181   740

             precision    recall  f1-score   support

          1       0.54      0.66      0.59      1640
          2       0.55      0.24      0.34      2198
          3       0.39      0.70      0.50      1060

avg / total       0.51      0.48      0.46      4898

RMSE:
0.8849727718184005
      __Prediction___
         1     2     3
    1  541   145    58
    2  217   220   201
    3   11    43   163

             precision    recall  f1-score   support

          1       0.70      0.73      0.72       744
          2       0.54      0.34      0.42       638
          3       0.39      0.75      0.51       217

avg / total       0.59      0.58      0.57      1599

RMSE:
0.7426942484742277
0.1
      __Prediction___
         1     2     3
    1 1081   268   291
    2  786   531   881
    3  139   180   741

             precision    recall  f1-score   support

       

             precision    recall  f1-score   support

          1       0.54      0.66      0.59      1640
          2       0.54      0.22      0.32      2198
          3       0.39      0.72      0.50      1060

avg / total       0.51      0.48      0.45      4898

RMSE:
0.886240724339699
      __Prediction___
         1     2     3
    1  552   121    71
    2  209   188   241
    3   11    34   172

             precision    recall  f1-score   support

          1       0.72      0.74      0.73       744
          2       0.55      0.29      0.38       638
          3       0.36      0.79      0.49       217

avg / total       0.60      0.57      0.56      1599

RMSE:
0.7638649625756222
1.2
      __Prediction___
         1     2     3
    1 1092   257   291
    2  799   486   913
    3  139   158   763

             precision    recall  f1-score   support

          1       0.54      0.67      0.60      1640
          2       0.54      0.22      0.31      2198
          3       0.3

             precision    recall  f1-score   support

          1       0.53      0.68      0.59      1640
          2       0.54      0.20      0.30      2198
          3       0.38      0.72      0.50      1060

avg / total       0.50      0.47      0.44      4898

RMSE:
0.893924870530357
      __Prediction___
         1     2     3
    1  556   105    83
    2  210   171   257
    3   11    29   177

             precision    recall  f1-score   support

          1       0.72      0.75      0.73       744
          2       0.56      0.27      0.36       638
          3       0.34      0.82      0.48       217

avg / total       0.60      0.57      0.55      1599

RMSE:
0.781669290236966
2.2
      __Prediction___
         1     2     3
    1 1109   230   301
    2  837   449   912
    3  145   158   757

             precision    recall  f1-score   support

          1       0.53      0.68      0.59      1640
          2       0.54      0.20      0.30      2198
          3       0.38

             precision    recall  f1-score   support

          1       0.53      0.68      0.60      1640
          2       0.53      0.19      0.28      2198
          3       0.39      0.72      0.50      1060

avg / total       0.50      0.47      0.43      4898

RMSE:
0.8920958663215764
      __Prediction___
         1     2     3
    1  560    87    97
    2  223   144   271
    3   11    22   184

             precision    recall  f1-score   support

          1       0.71      0.75      0.73       744
          2       0.57      0.23      0.32       638
          3       0.33      0.85      0.48       217

avg / total       0.60      0.56      0.53      1599

RMSE:
0.804536854170506
3.2
      __Prediction___
         1     2     3
    1 1126   223   291
    2  863   412   923
    3  143   150   767

             precision    recall  f1-score   support

          1       0.53      0.69      0.60      1640
          2       0.52      0.19      0.28      2198
          3       0.3

             precision    recall  f1-score   support

          1       0.53      0.70      0.60      1640
          2       0.54      0.18      0.27      2198
          3       0.39      0.73      0.51      1060

avg / total       0.50      0.47      0.43      4898

RMSE:
0.8943815371606701
      __Prediction___
         1     2     3
    1  576    72    96
    2  245   109   284
    3   13    27   177

             precision    recall  f1-score   support

          1       0.69      0.77      0.73       744
          2       0.52      0.17      0.26       638
          3       0.32      0.82      0.46       217

avg / total       0.57      0.54      0.50      1599

RMSE:
0.8157302770696206
4.2
      __Prediction___
         1     2     3
    1 1151   195   294
    2  893   379   926
    3  151   131   778

             precision    recall  f1-score   support

          1       0.52      0.70      0.60      1640
          2       0.54      0.17      0.26      2198
          3       0.

...

C:\Users\Michael\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Michael\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Michael\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Michael\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, wa

             precision    recall  f1-score   support

          1       0.49      0.76      0.60      1640
          2       0.48      0.07      0.12      2198
          3       0.39      0.75      0.51      1060

avg / total       0.46      0.45      0.36      4898

RMSE:
0.9266702995229568
      __Prediction___
         1     2     3
    1  727    17     0
    2  608    30     0
    3  198    19     0

             precision    recall  f1-score   support

          1       0.47      0.98      0.64       744
          2       0.45      0.05      0.09       638
          3       0.00      0.00      0.00       217

avg / total       0.40      0.47      0.33      1599

RMSE:
0.9476609564106726
9.2
      __Prediction___
         1     2     3
    1 1251    96   293
    2 1088   148   962
    3  212    51   797

             precision    recall  f1-score   support

          1       0.49      0.76      0.60      1640
          2       0.50      0.07      0.12      2198
          3       0.

C:\Users\Michael\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Michael\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Michael\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Michael\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, wa

             precision    recall  f1-score   support

          1       0.49      0.77      0.60      1640
          2       0.54      0.07      0.12      2198
          3       0.39      0.75      0.51      1060

avg / total       0.49      0.45      0.37      4898

RMSE:
0.9293103911960597
      __Prediction___
         1     2     3
    1  744     0     0
    2  638     0     0
    3  217     0     0

             precision    recall  f1-score   support

          1       0.47      1.00      0.64       744
          2       0.00      0.00      0.00       638
          3       0.00      0.00      0.00       217

avg / total       0.22      0.47      0.30      1599

RMSE:
0.970483719160565
9.6
      __Prediction___
         1     2     3
    1 1259    89   292
    2 1092   153   953
    3  222    39   799

             precision    recall  f1-score   support

          1       0.49      0.77      0.60      1640
          2       0.54      0.07      0.12      2198
          3       0.3

C:\Users\Michael\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Michael\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Michael\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Michael\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, wa

In [39]:
for t in range (0,10000,100):
    t = t/1000
    print(t)
    print("#____________________red STD normed____________________#")
    run_kncs_report(red_norm.values, red_targetclass)
    print("#____________________red MinMax normed____________________#")
    run_kncs_report(mm_red_norm.values, red_targetclass)
    print("#____________________red log normed____________________#")
    run_kncs_report(log_red_norm.values, red_targetclass)

0.0
#____________________red STD normed____________________#
      __Prediction___
         1     2     3
    1  541   145    58
    2  217   220   201
    3   11    43   163

             precision    recall  f1-score   support

          1       0.70      0.73      0.72       744
          2       0.54      0.34      0.42       638
          3       0.39      0.75      0.51       217

avg / total       0.59      0.58      0.57      1599

RMSE:
0.7426942484742277
#____________________red MinMax normed____________________#
      __Prediction___
         1     2     3
    1  559   113    72
    2  243   189   206
    3   12    44   161

             precision    recall  f1-score   support

          1       0.69      0.75      0.72       744
          2       0.55      0.30      0.38       638
          3       0.37      0.74      0.49       217

avg / total       0.59      0.57      0.55      1599

RMSE:
0.7675403565118231
#____________________red log normed____________________#
      

             precision    recall  f1-score   support

          1       0.69      0.75      0.72       744
          2       0.55      0.30      0.38       638
          3       0.37      0.74      0.49       217

avg / total       0.59      0.57      0.55      1599

RMSE:
0.7675403565118231
#____________________red log normed____________________#
      __Prediction___
         1     2     3
    1  430    81   233
    2  289   100   249
    3   66    23   128

             precision    recall  f1-score   support

          1       0.55      0.58      0.56       744
          2       0.49      0.16      0.24       638
          3       0.21      0.59      0.31       217

avg / total       0.48      0.41      0.40      1599

RMSE:
1.0721326493308097
0.7
#____________________red STD normed____________________#
      __Prediction___
         1     2     3
    1  541   145    58
    2  217   220   201
    3   11    43   163

             precision    recall  f1-score   support

          1 

             precision    recall  f1-score   support

          1       0.70      0.73      0.72       744
          2       0.54      0.34      0.42       638
          3       0.39      0.75      0.51       217

avg / total       0.59      0.58      0.57      1599

RMSE:
0.7426942484742277
#____________________red MinMax normed____________________#
      __Prediction___
         1     2     3
    1  559   113    72
    2  243   189   206
    3   12    44   161

             precision    recall  f1-score   support

          1       0.69      0.75      0.72       744
          2       0.55      0.30      0.38       638
          3       0.37      0.74      0.49       217

avg / total       0.59      0.57      0.55      1599

RMSE:
0.7675403565118231
#____________________red log normed____________________#
      __Prediction___
         1     2     3
    1  430    81   233
    2  289   100   249
    3   66    23   128

             precision    recall  f1-score   support

          1  

             precision    recall  f1-score   support

          1       0.70      0.73      0.72       744
          2       0.54      0.34      0.42       638
          3       0.39      0.75      0.51       217

avg / total       0.59      0.58      0.57      1599

RMSE:
0.7426942484742277
#____________________red MinMax normed____________________#
      __Prediction___
         1     2     3
    1  559   113    72
    2  243   189   206
    3   12    44   161

             precision    recall  f1-score   support

          1       0.69      0.75      0.72       744
          2       0.55      0.30      0.38       638
          3       0.37      0.74      0.49       217

avg / total       0.59      0.57      0.55      1599

RMSE:
0.7675403565118231
#____________________red log normed____________________#
      __Prediction___
         1     2     3
    1  430    81   233
    2  289   100   249
    3   66    23   128

             precision    recall  f1-score   support

          1  

             precision    recall  f1-score   support

          1       0.70      0.73      0.72       744
          2       0.54      0.34      0.42       638
          3       0.39      0.75      0.51       217

avg / total       0.59      0.58      0.57      1599

RMSE:
0.7426942484742277
#____________________red MinMax normed____________________#
      __Prediction___
         1     2     3
    1  559   113    72
    2  243   189   206
    3   12    44   161

             precision    recall  f1-score   support

          1       0.69      0.75      0.72       744
          2       0.55      0.30      0.38       638
          3       0.37      0.74      0.49       217

avg / total       0.59      0.57      0.55      1599

RMSE:
0.7675403565118231
#____________________red log normed____________________#
      __Prediction___
         1     2     3
    1  430    81   233
    2  289   100   249
    3   66    23   128

             precision    recall  f1-score   support

          1  

...

             precision    recall  f1-score   support

          1       0.55      0.58      0.56       744
          2       0.49      0.16      0.24       638
          3       0.21      0.59      0.31       217

avg / total       0.48      0.41      0.40      1599

RMSE:
1.0721326493308097
6.7
#____________________red STD normed____________________#
      __Prediction___
         1     2     3
    1  541   145    58
    2  217   220   201
    3   11    43   163

             precision    recall  f1-score   support

          1       0.70      0.73      0.72       744
          2       0.54      0.34      0.42       638
          3       0.39      0.75      0.51       217

avg / total       0.59      0.58      0.57      1599

RMSE:
0.7426942484742277
#____________________red MinMax normed____________________#
      __Prediction___
         1     2     3
    1  559   113    72
    2  243   189   206
    3   12    44   161

             precision    recall  f1-score   support

         

             precision    recall  f1-score   support

          1       0.70      0.73      0.72       744
          2       0.54      0.34      0.42       638
          3       0.39      0.75      0.51       217

avg / total       0.59      0.58      0.57      1599

RMSE:
0.7426942484742277
#____________________red MinMax normed____________________#
      __Prediction___
         1     2     3
    1  559   113    72
    2  243   189   206
    3   12    44   161

             precision    recall  f1-score   support

          1       0.69      0.75      0.72       744
          2       0.55      0.30      0.38       638
          3       0.37      0.74      0.49       217

avg / total       0.59      0.57      0.55      1599

RMSE:
0.7675403565118231
#____________________red log normed____________________#
      __Prediction___
         1     2     3
    1  430    81   233
    2  289   100   249
    3   66    23   128

             precision    recall  f1-score   support

          1  

             precision    recall  f1-score   support

          1       0.70      0.73      0.72       744
          2       0.54      0.34      0.42       638
          3       0.39      0.75      0.51       217

avg / total       0.59      0.58      0.57      1599

RMSE:
0.7426942484742277
#____________________red MinMax normed____________________#
      __Prediction___
         1     2     3
    1  559   113    72
    2  243   189   206
    3   12    44   161

             precision    recall  f1-score   support

          1       0.69      0.75      0.72       744
          2       0.55      0.30      0.38       638
          3       0.37      0.74      0.49       217

avg / total       0.59      0.57      0.55      1599

RMSE:
0.7675403565118231
#____________________red log normed____________________#
      __Prediction___
         1     2     3
    1  430    81   233
    2  289   100   249
    3   66    23   128

             precision    recall  f1-score   support

          1  

             precision    recall  f1-score   support

          1       0.69      0.75      0.72       744
          2       0.55      0.30      0.38       638
          3       0.37      0.74      0.49       217

avg / total       0.59      0.57      0.55      1599

RMSE:
0.7675403565118231
#____________________red log normed____________________#
      __Prediction___
         1     2     3
    1  430    81   233
    2  289   100   249
    3   66    23   128

             precision    recall  f1-score   support

          1       0.55      0.58      0.56       744
          2       0.49      0.16      0.24       638
          3       0.21      0.59      0.31       217

avg / total       0.48      0.41      0.40      1599

RMSE:
1.0721326493308097
8.7
#____________________red STD normed____________________#
      __Prediction___
         1     2     3
    1  541   145    58
    2  217   220   201
    3   11    43   163

             precision    recall  f1-score   support

          1 

             precision    recall  f1-score   support

          1       0.69      0.75      0.72       744
          2       0.55      0.30      0.38       638
          3       0.37      0.74      0.49       217

avg / total       0.59      0.57      0.55      1599

RMSE:
0.7675403565118231
#____________________red log normed____________________#
      __Prediction___
         1     2     3
    1  430    81   233
    2  289   100   249
    3   66    23   128

             precision    recall  f1-score   support

          1       0.55      0.58      0.56       744
          2       0.49      0.16      0.24       638
          3       0.21      0.59      0.31       217

avg / total       0.48      0.41      0.40      1599

RMSE:
1.0721326493308097
9.3
#____________________red STD normed____________________#
      __Prediction___
         1     2     3
    1  541   145    58
    2  217   220   201
    3   11    43   163

             precision    recall  f1-score   support

          1 

In [40]:
def run_knnw_report(inputs, targets):
    print("_______________________________________________________________________________________________")
    print("KNN with inverse distance as weight:")
    for n_neighbour in range(1,21):
        print(str(n_neighbour) + " neighbours:")
        knn_estimator = KNeighborsClassifier(n_neighbour,weights='distance')
        print(str(n_neighbour)+ " neighbours//")
        predicted = cross_val_predict(knn_estimator,inputs,targets,cv=cv)
        print(confusion_matrix_report(targets,predicted))
        print(classification_report(targets,predicted))
        try:
            print("RMSE:")
            print(root_mean_squared_error(targets,predicted))
        except(Error):
            pass

In [41]:
print("#____________________red STD normed____________________#")
run_knnw_report(f_red_norm.values, red_targetclass)

#____________________red STD normed____________________#
_______________________________________________________________________________________________
KNN with inverse distance as weight:
1 neighbours:
1 neighbours//
      __Prediction___
         1     2     3
    1  567   165    12
    2  147   417    74
    3   18    69   130

             precision    recall  f1-score   support

          1       0.77      0.76      0.77       744
          2       0.64      0.65      0.65       638
          3       0.60      0.60      0.60       217

avg / total       0.70      0.70      0.70      1599

RMSE:
0.5996663654430289
2 neighbours:
2 neighbours//
      __Prediction___
         1     2     3
    1  567   165    12
    2  148   416    74
    3   18    69   130

             precision    recall  f1-score   support

          1       0.77      0.76      0.77       744
          2       0.64      0.65      0.65       638
          3       0.60      0.60      0.60       217

avg / total    

      __Prediction___
         1     2     3
    1  606   134     4
    2  137   457    44
    3   11    78   128

             precision    recall  f1-score   support

          1       0.80      0.81      0.81       744
          2       0.68      0.72      0.70       638
          3       0.73      0.59      0.65       217

avg / total       0.75      0.74      0.74      1599

RMSE:
0.5322612739903859


In [42]:
print("#____________________red MinMax normed____________________#")
run_knnw_report(f_mm_red.values, red_targetclass)

#____________________red MinMax normed____________________#
_______________________________________________________________________________________________
KNN with inverse distance as weight:
1 neighbours:
1 neighbours//
      __Prediction___
         1     2     3
    1  567   163    14
    2  142   420    76
    3   15    66   136

             precision    recall  f1-score   support

          1       0.78      0.76      0.77       744
          2       0.65      0.66      0.65       638
          3       0.60      0.63      0.61       217

avg / total       0.70      0.70      0.70      1599

RMSE:
0.5933759848629978
2 neighbours:
2 neighbours//
      __Prediction___
         1     2     3
    1  567   163    14
    2  142   420    76
    3   15    66   136

             precision    recall  f1-score   support

          1       0.78      0.76      0.77       744
          2       0.65      0.66      0.65       638
          3       0.60      0.63      0.61       217

avg / total 

In [43]:
print("#____________________red log normed____________________#")
run_knnw_report(f_log_red.values, red_targetclass)

#____________________red log normed____________________#
_______________________________________________________________________________________________
KNN with inverse distance as weight:
1 neighbours:
1 neighbours//
      __Prediction___
         1     2     3
    1  568   160    16
    2  154   402    82
    3   16    66   135

             precision    recall  f1-score   support

          1       0.77      0.76      0.77       744
          2       0.64      0.63      0.64       638
          3       0.58      0.62      0.60       217

avg / total       0.69      0.69      0.69      1599

RMSE:
0.6074377440388866
2 neighbours:
2 neighbours//
      __Prediction___
         1     2     3
    1  568   160    16
    2  154   402    82
    3   16    66   135

             precision    recall  f1-score   support

          1       0.77      0.76      0.77       744
          2       0.64      0.63      0.64       638
          3       0.58      0.62      0.60       217

avg / total    

In [44]:
print("#____________________white STD normed____________________#")
run_knnw_report(white_norm.values, white_targetclass)

#____________________white STD normed____________________#
_______________________________________________________________________________________________
KNN with inverse distance as weight:
1 neighbours:
1 neighbours//
      __Prediction___
         1     2     3
    1 1156   416    68
    2  381  1538   279
    3   59   276   725

             precision    recall  f1-score   support

          1       0.72      0.70      0.71      1640
          2       0.69      0.70      0.69      2198
          3       0.68      0.68      0.68      1060

avg / total       0.70      0.70      0.70      4898

RMSE:
0.6162360225133208
2 neighbours:
2 neighbours//
      __Prediction___
         1     2     3
    1 1156   416    68
    2  381  1538   279
    3   59   276   725

             precision    recall  f1-score   support

          1       0.72      0.70      0.71      1640
          2       0.69      0.70      0.69      2198
          3       0.68      0.68      0.68      1060

avg / total  

      __Prediction___
         1     2     3
    1 1178   423    39
    2  326  1662   210
    3   34   320   706

             precision    recall  f1-score   support

          1       0.77      0.72      0.74      1640
          2       0.69      0.76      0.72      2198
          3       0.74      0.67      0.70      1060

avg / total       0.73      0.72      0.72      4898

RMSE:
0.5663419112812177


In [45]:
print("#____________________white MinMax normed____________________#")
run_knnw_report(mm_white_norm.values, white_targetclass)

#____________________white MinMax normed____________________#
_______________________________________________________________________________________________
KNN with inverse distance as weight:
1 neighbours:
1 neighbours//
      __Prediction___
         1     2     3
    1 1163   408    69
    2  375  1528   295
    3   60   277   723

             precision    recall  f1-score   support

          1       0.73      0.71      0.72      1640
          2       0.69      0.70      0.69      2198
          3       0.67      0.68      0.67      1060

avg / total       0.70      0.70      0.70      4898

RMSE:
0.6180555396250805
2 neighbours:
2 neighbours//
      __Prediction___
         1     2     3
    1 1163   408    69
    2  375  1528   295
    3   60   277   723

             precision    recall  f1-score   support

          1       0.73      0.71      0.72      1640
          2       0.69      0.70      0.69      2198
          3       0.67      0.68      0.67      1060

avg / tota

      __Prediction___
         1     2     3
    1 1157   427    56
    2  323  1635   240
    3   24   318   718

             precision    recall  f1-score   support

          1       0.77      0.71      0.74      1640
          2       0.69      0.74      0.71      2198
          3       0.71      0.68      0.69      1060

avg / total       0.72      0.72      0.72      4898

RMSE:
0.5765245558476275


In [46]:
print("#____________________white log normed____________________#")
run_knnw_report(log_white_norm.values, white_targetclass)

#____________________white log normed____________________#
_______________________________________________________________________________________________
KNN with inverse distance as weight:
1 neighbours:
1 neighbours//
      __Prediction___
         1     2     3
    1 1139   412    89
    2  416  1464   318
    3   67   304   689

             precision    recall  f1-score   support

          1       0.70      0.69      0.70      1640
          2       0.67      0.67      0.67      2198
          3       0.63      0.65      0.64      1060

avg / total       0.67      0.67      0.67      4898

RMSE:
0.6507212444783992
2 neighbours:
2 neighbours//
      __Prediction___
         1     2     3
    1 1139   412    89
    2  416  1464   318
    3   67   304   689

             precision    recall  f1-score   support

          1       0.70      0.69      0.70      1640
          2       0.67      0.67      0.67      2198
          3       0.63      0.65      0.64      1060

avg / total  

             precision    recall  f1-score   support

          1       0.74      0.68      0.71      1640
          2       0.66      0.75      0.70      2198
          3       0.71      0.62      0.66      1060

avg / total       0.70      0.70      0.70      4898

RMSE:
0.5992713692047622
20 neighbours:
20 neighbours//
      __Prediction___
         1     2     3
    1 1120   472    48
    2  341  1639   218
    3   40   365   655

             precision    recall  f1-score   support

          1       0.75      0.68      0.71      1640
          2       0.66      0.75      0.70      2198
          3       0.71      0.62      0.66      1060

avg / total       0.70      0.70      0.70      4898

RMSE:
0.5973946428704721


In [47]:
run_knnw_report(white_input.values,white_targetclass)

_______________________________________________________________________________________________
KNN with inverse distance as weight:
1 neighbours:
1 neighbours//
      __Prediction___
         1     2     3
    1 1052   450   138
    2  437  1435   326
    3  107   325   628

             precision    recall  f1-score   support

          1       0.66      0.64      0.65      1640
          2       0.65      0.65      0.65      2198
          3       0.58      0.59      0.58      1060

avg / total       0.64      0.64      0.64      4898

RMSE:
0.716998872108698
2 neighbours:
2 neighbours//
      __Prediction___
         1     2     3
    1 1052   450   138
    2  437  1435   326
    3  107   325   628

             precision    recall  f1-score   support

          1       0.66      0.64      0.65      1640
          2       0.65      0.65      0.65      2198
          3       0.58      0.59      0.58      1060

avg / total       0.64      0.64      0.64      4898

RMSE:
0.71699887210

      __Prediction___
         1     2     3
    1 1043   521    76
    2  395  1603   200
    3   84   415   561

             precision    recall  f1-score   support

          1       0.69      0.64      0.66      1640
          2       0.63      0.73      0.68      2198
          3       0.67      0.53      0.59      1060

avg / total       0.66      0.65      0.65      4898

RMSE:
0.6657643274078571


In [48]:
run_knnw_report(red_input.values,red_targetclass)

_______________________________________________________________________________________________
KNN with inverse distance as weight:
1 neighbours:
1 neighbours//
      __Prediction___
         1     2     3
    1  518   188    38
    2  181   398    59
    3   28    75   114

             precision    recall  f1-score   support

          1       0.71      0.70      0.70       744
          2       0.60      0.62      0.61       638
          3       0.54      0.53      0.53       217

avg / total       0.65      0.64      0.64      1599

RMSE:
0.6925855880307988
2 neighbours:
2 neighbours//
      __Prediction___
         1     2     3
    1  518   188    38
    2  181   398    59
    3   28    75   114

             precision    recall  f1-score   support

          1       0.71      0.70      0.70       744
          2       0.60      0.62      0.61       638
          3       0.54      0.53      0.53       217

avg / total       0.65      0.64      0.64      1599

RMSE:
0.6925855880

In [49]:
run_knn_report(f_log_white.values,white_targetclass)
run_knnw_report(f_log_white.values,white_targetclass)

_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1 1081   461    98
    2  439  1413   346
    3   98   327   635

             precision    recall  f1-score   support

          1       0.67      0.66      0.66      1640
          2       0.64      0.64      0.64      2198
          3       0.59      0.60      0.59      1060

avg / total       0.64      0.64      0.64      4898

RMSE:
0.6936979336808926
2 neighbours:
      __Prediction___
         1     2     3
    1 1278   319    43
    2  810  1228   160
    3  199   520   341

             precision    recall  f1-score   support

          1       0.56      0.78      0.65      1640
          2       0.59      0.56      0.58      2198
          3       0.63      0.32      0.43      1060

avg / total       0.59      0.58      0.57      4898

RMSE:
0.7529715191265614
3 neighbours:
      __Prediction___
        

      __Prediction___
         1     2     3
    1 1081   461    98
    2  439  1413   346
    3   98   327   635

             precision    recall  f1-score   support

          1       0.67      0.66      0.66      1640
          2       0.64      0.64      0.64      2198
          3       0.59      0.60      0.59      1060

avg / total       0.64      0.64      0.64      4898

RMSE:
0.6936979336808926
2 neighbours:
2 neighbours//
      __Prediction___
         1     2     3
    1 1095   451    94
    2  459  1399   340
    3  101   336   623

             precision    recall  f1-score   support

          1       0.66      0.67      0.66      1640
          2       0.64      0.64      0.64      2198
          3       0.59      0.59      0.59      1060

avg / total       0.64      0.64      0.64      4898

RMSE:
0.6950210844864835
3 neighbours:
3 neighbours//
      __Prediction___
         1     2     3
    1 1106   437    97
    2  437  1434   327
    3   78   359   623

           

      __Prediction___
         1     2     3
    1 1147   439    54
    2  361  1605   232
    3   38   396   626

             precision    recall  f1-score   support

          1       0.74      0.70      0.72      1640
          2       0.66      0.73      0.69      2198
          3       0.69      0.59      0.63      1060

avg / total       0.69      0.69      0.69      4898

RMSE:
0.6055413096266453


In [50]:
run_knn_report(f_mm_red.values,red_targetclass)
run_knnw_report(f_mm_red.values,red_targetclass)

_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1  567   163    14
    2  142   420    76
    3   15    66   136

             precision    recall  f1-score   support

          1       0.78      0.76      0.77       744
          2       0.65      0.66      0.65       638
          3       0.60      0.63      0.61       217

avg / total       0.70      0.70      0.70      1599

RMSE:
0.5933759848629978
2 neighbours:
      __Prediction___
         1     2     3
    1  639   103     2
    2  281   325    32
    3   34   125    58

             precision    recall  f1-score   support

          1       0.67      0.86      0.75       744
          2       0.59      0.51      0.55       638
          3       0.63      0.27      0.38       217

avg / total       0.63      0.64      0.62      1599

RMSE:
0.6545171850042718
3 neighbours:
      __Prediction___
        

             precision    recall  f1-score   support

          1       0.78      0.76      0.77       744
          2       0.65      0.66      0.65       638
          3       0.60      0.63      0.61       217

avg / total       0.70      0.70      0.70      1599

RMSE:
0.5933759848629978
2 neighbours:
2 neighbours//
      __Prediction___
         1     2     3
    1  567   163    14
    2  142   420    76
    3   15    66   136

             precision    recall  f1-score   support

          1       0.78      0.76      0.77       744
          2       0.65      0.66      0.65       638
          3       0.60      0.63      0.61       217

avg / total       0.70      0.70      0.70      1599

RMSE:
0.5933759848629978
3 neighbours:
3 neighbours//
      __Prediction___
         1     2     3
    1  575   158    11
    2  139   426    73
    3   18    69   130

             precision    recall  f1-score   support

          1       0.79      0.77      0.78       744
          2       0